# Worksheet 08

Name: Yu Han
UID:  U74808634

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [74]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.81743029005411, 4.834885507343804, 4.62998119269431, 4.768123051499574, 6.788916376415189, 5.708567911969055, 4.26847312353234, 4.367685864828221, 5.437513334686686, 6.501266672787494]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [75]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[7.767470960924775, 10.252541531790483, 8.641110502615573, 7.460093014314079, 7.7768733678172985, 7.937692760397383, 8.105902487283371, 7.7478706039832455, 7.513387207630212, 8.68638896763101]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [76]:
data = []
a = 0
b = 0
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
        a+=1
    else:
        p2 = s2.pop()
        data.append(p2)
        b+=1
print(data)
print(a)
print(b)

[6.501266672787494, 8.68638896763101, 5.437513334686686, 4.367685864828221, 4.26847312353234, 7.513387207630212, 5.708567911969055, 7.7478706039832455, 6.788916376415189, 4.768123051499574]
7
3


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

1. the centroid is the mean: 5, 8
2. variance = 1

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [77]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] != 0, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2)/(len(s1) + len(s2))]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.437513334686686, 4.367685864828221, 4.26847312353234, 5.708567911969055, 4.768123051499574]
[6.501266672787494, 8.68638896763101, 7.513387207630212, 7.7478706039832455, 6.788916376415189]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 4.910072657303175,  mean_2 = 7.447565965689431
var_1 = 0.3283542849713451,  var_2 = 0.5916998750017572


c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


They are close to the true values.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [78]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( prob_s[0] * pdf_i[0]/ prob_x )
    prob_s1_x.append( prob_s[1] * pdf_i[1]/ prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  6.501266672787494
probability of observing that point if it came from cluster 0 =  9.665243073659768e-06
probability of observing that point if it came from cluster 1 =  0.18767454441618817
point =  8.68638896763101
probability of observing that point if it came from cluster 0 =  2.3076143588868091e-29
probability of observing that point if it came from cluster 1 =  0.07532785333261414
point =  5.437513334686686
probability of observing that point if it came from cluster 0 =  0.3344052666054206
probability of observing that point if it came from cluster 1 =  0.0021032663002348026
point =  4.367685864828221
probability of observing that point if it came from cluster 0 =  0.3105053020960391
probability of observing that point if it came from cluster 1 =  8.821366658427761e-07
point =  4.26847312353234
probability of observing that point if it came from cluster 0 =  0.18008862011975363
probability of observing that point if it came from cluster 1 =  3.6340674551282397e-07
point =

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [79]:
prob_s = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
       sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.4869317470658016,  P(S_2) = 0.5130682529341984
mean_1 = 4.889004570554515,  mean_2 = 7.402928855609056
var_1 = 0.320095573844894,  var_2 = 0.6529461120863479



Means are close to what I got from K means, yet probabilities and variances are far from what I got, especially the probabilities.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [80]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( prob_s[0] * pdf_i[0]/ prob_x )
    prob_s1_x.append( prob_s[1] * pdf_i[1]/ prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  6.501266672787494
probability of observing that point if it came from cluster 0 =  3.861116375857089e-06
probability of observing that point if it came from cluster 1 =  0.23547769653345285
point =  8.68638896763101
probability of observing that point if it came from cluster 0 =  3.426682497490001e-31
probability of observing that point if it came from cluster 1 =  0.08851727550743854
point =  5.437513334686686
probability of observing that point if it came from cluster 0 =  0.28708474728091576
probability of observing that point if it came from cluster 1 =  0.006585086165671773
point =  4.367685864828221
probability of observing that point if it came from cluster 0 =  0.3308705405526826
probability of observing that point if it came from cluster 1 =  1.2408638865519034e-05
point =  4.26847312353234
probability of observing that point if it came from cluster 0 =  0.19035728395064797
probability of observing that point if it came from cluster 1 =  6.052802462963312e-06
point = 

Most of the points are from S_1.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [81]:
assignments = [[], []]
for x in zip(data, prob_s0_x):
    prob = x[1]
    data = x[0]
    if random.random() <= prob:
        assignments[0].append(data)
    else:
        assignments[1].append(data)
print(f"points in cluster 0 : {assignments[0]}")
print(f"points in cluster 1 : {assignments[1]}")

points in cluster 0 : [5.437513334686686, 4.367685864828221, 4.26847312353234, 5.708567911969055, 4.768123051499574]
points in cluster 1 : [6.501266672787494, 8.68638896763101, 7.513387207630212, 7.7478706039832455, 6.788916376415189]
